# 1. Introduction

This project is an exploration of creating a chess application from first principles, starting with a basic implementation in Python. The goal is to develop a chess engine capable of storing and processing game states as an 8x8 NumPy array, along with relevant metadata for each turn. Here's the roadmap:

1. **Initial Development**:
   - Create a functioning chess game in Python.
   - Simulate and store games with turn-by-turn data.

2. **Data Integration**:
   - Leverage game data from lichess.org.
   - Engineer features to train a neural network classifier capable of predicting optimal moves.

3. **Objective**:
   - Train the model to play chess effectively, aiming to outperform a casual player.
   - If successful, port the application to Android with added Bluetooth networking for offline gameplay.

This notebook is built in Jupyter, combining markdown and Python code cells in a shared execution environment. You can run cells interactively or execute everything sequentially using the **Run All** option.

# 2. The Code

## 2.1 Imports, constants and enums

I like to keep all my constants, enums and imports all up front, just an organizational quirk from previous languages.
Here I'm breaking things into Enums instead of just having a bunch of global variables, mostly just for readability, but it also lets me pass information back with a MoveResult.

For MoveResult, we have various error conditions and turn states. 
For Color, we have black, white and empty. 
For PieceType we have each of the pieces plus empty.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import unittest

from enum import Enum
from typing import Optional, Tuple, List, Dict
from numpy import ndarray
from matplotlib.patches import Rectangle
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.widgets import Button

# Enums for game mechanics
class MoveResult(Enum):
    """
    Represents the result of a move in the game.
    """
    ERROR_CHECK = 0
    ERROR_NO_PIECE = 1
    ERROR_WRONG_COLOR = 2
    ERROR_CANNOT_REACH = 3
    ERROR_CANNOT_CASTLE = 4
    WHITE_TURN = 5
    BLACK_TURN = 6
    BLACK_TURN_CHECK = 7
    WHITE_TURN_CHECK = 8
    BLACK_WINS = 9
    WHITE_WINS = 10
    ERROR_INVALID_MOVE = 11

class Color(Enum):
    """
    Represents the color of a piece or an empty square.
    """
    EMPTY = 0
    WHITE = 1
    BLACK = 2

class PieceType(Enum):
    """
    Represents the type of a chess piece or an empty square.
    """
    EMPTY = 0
    PAWN = 1
    ROOK = 2
    KNIGHT = 3
    BISHOP = 4
    QUEEN = 5
    KING = 6

# Constants for indexing move and piece data
MOVE_START_ROW, MOVE_START_COL, MOVE_END_ROW, MOVE_END_COL = 0, 1, 2, 3
TYPE, COLOR = 0, 1
LOCATION_ROW, LOCATION_COLUMN = 0, 1
WHITE_QUEEN_SIDE, WHITE_KING_SIDE, BLACK_QUEEN_SIDE, BLACK_KING_SIDE = 0, 1, 2, 3

# Move directions for different pieces
BISHOP_MOVE_DIRECTIONS: List[Tuple[int, int]] = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
ROOK_MOVE_DIRECTIONS: List[Tuple[int, int]] = [(1, 0), (-1, 0), (0, 1), (0, -1)]
ROYAL_MOVE_DIRECTIONS: List[Tuple[int, int]] = [
    (1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)
]

## 2.2 The Chess Pieces

In this section, we define the chess pieces. While their primary purpose is readability and facilitating conversion of the board into a NumPy array, the game logic resides in the broader `Game` and `Board` classes.

### Design Considerations:
- **Minimal Responsibilities**:
  - Each piece is defined by its `PieceType` and `Color`.
  - No methods like `get_possible_moves()` are included here, as determining possible moves depends on the state of the board and other pieces, which is outside the scope of individual pieces.
  
- **Potential Extensions**:
  - A `has_moved()` method or property could be added to track piece movement, but this functionality is kept within the game logic for modularity. This approach ensures that the pieces and board can potentially be reused in non-standard chess scenarios.

In [ ]:
class ChessPiece:
    """
    Represents a generic chess piece with a color and type.
    """
    def __init__(self, color: Color, piece_type: PieceType, row: int, col: int) -> None:
        """
        Initializes a chess piece.

        Args:
            color (Color): The color of the piece (WHITE, BLACK, or EMPTY).
            piece_type (PieceType): The type of the chess piece.
        """
        self.color: Color = color
        self.piece_type: PieceType = piece_type
        self.row = row
        self.col = col

    def to_numpy(self) -> ndarray:
        """
        Converts the chess piece to a NumPy array representation.

        Returns:
            ndarray: A 2-element array [color value, piece type value].
        """
        return np.array([self.color.value, self.piece_type.value])

    def move_to(self, row: int, col: int) -> None:
        self.row = row
        self.col = col

# Subclasses for specific chess pieces
class Pawn(ChessPiece):
    """
    Represents a pawn chess piece.
    """
    def __init__(self, color: Color, row: int, col: int) -> None:
        super().__init__(color, PieceType.PAWN, row, col)

class Rook(ChessPiece):
    """
    Represents a rook chess piece.
    """
    def __init__(self, color: Color, row: int, col: int) -> None:
        super().__init__(color, PieceType.ROOK, row, col)

class Knight(ChessPiece):
    """
    Represents a knight chess piece.
    """
    def __init__(self, color: Color, row: int, col: int) -> None:
        super().__init__(color, PieceType.KNIGHT, row, col)

class Bishop(ChessPiece):
    """
    Represents a bishop chess piece.
    """
    def __init__(self, color: Color, row: int, col: int) -> None:
        super().__init__(color, PieceType.BISHOP, row, col)

class Queen(ChessPiece):
    """
    Represents a queen chess piece.
    """
    def __init__(self, color: Colo, row: int, col: int) -> None:
        super().__init__(color, PieceType.QUEEN, row, col)

class King(ChessPiece):
    """
    Represents a king chess piece.
    """
    def __init__(self, color: Color, row: int, col: int) -> None:
        super().__init__(color, PieceType.KING, row, col)


## 2.3 The Board

The board is represented as its own class, responsible for maintaining the current state of the pieces on the board and providing information about possible moves or board status. Its design focuses on simplicity, deferring more complex game logic (e.g., turn tracking, move validation for special rules) to the `ChessGame` class.

### Responsibilities:
- **Board State**:
  - Tracks the pieces occupying each square.
  - Provides methods to query board information, such as whether a player is in check or potential moves for a piece.
  
- **Stateless Functionality**:
  - The board has no memory of move history or game state (e.g., en passant, castling legality, turn tracking). These are managed by the `ChessGame` class.
  - This design ensures the board can be used for various scenarios, including custom setups or debugging invalid configurations.

### Key Methods:
1. **`__init__(self)`**:
   - Initializes the board with a standard chess setup. In the future, this could accept a custom configuration for saved games, custom scenarios or tests.

2. **`to_numpy(self)`**:
   - Converts the board to a NumPy array representation for further processing or analysis.

3. **`piece_at(self, location: Tuple[int, int]) -> Optional[ChessPiece]`**:
   - Returns the piece at a specified location, or `None` if the square is empty.

4. **`get_possible_moves(self, location: Tuple[int, int]) -> List[Tuple[int, int]]`**:
   - Provides a list of potential moves for a piece at the specified location.
   - Does not account for check, en passant or castling logic; it only evaluates raw movement rules.

5. **`is_in_check(self, color: Color) -> bool`**:
   - Checks if the specified player (`WHITE` or `BLACK`) is in check.

6. **`preview_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> Tuple[MoveResult, 'Board']`**:
   - Simulates a move, returning a new board and a `MoveResult`.
   - The board state remains unchanged, allowing analysis of hypothetical moves.

7. **`make_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> Tuple[MoveResult, 'Board']`**:
   - Executes a move, updating the board state and returning a `MoveResult`.
   - Finalizes the move, unlike `preview_move`.

### Design Philosophy:
The board is intentionally kept "dumb," offloading the majority of logic (e.g., turn tracking, advanced move validation) to the `ChessGame` class. This modular design allows the board to handle raw functionality while remaining reusable for tests or custom scenarios, including invalid setups or debugging purposes.

In [ ]:
class ChessBoard:
    """
    Represents the chessboard and provides methods to interact with its state.
    """
    def __init__(self, board = None, en_passant = None, castle_possibilities = (True, True, True, True)) -> None:
        """
        Initializes the chessboard with the standard setup of pieces.
        """
        self.en_passant = en_passant
        self.castle_possibilities = castle_possibilities
        if board:
            self.board = board
        else:
            self.board: List[List[Optional[ChessPiece]]] = []
            self.board: ndarray = np.empty((8, 8), dtype=object)
            for row in range(0, 8):
                self.board[row] = []
    
            # Place pawns
            for col in range(8):
                self.board[1][col] = Pawn(Color.WHITE, 1, col)
                self.board[6][col] = Pawn(Color.BLACK, 6, col)
    
            # Place rooks
            self.board[0][0] = [Rook(Color.WHITE, 0, 0)]
            self.board[0][7] = [Rook(Color.WHITE, 0, 7)]
            self.board[7][0] = [Rook(Color.BLACK, 7, 0)]
            self.board[7][7] = [Rook(Color.BLACK, 7, 7)]
    
            # Place knights
            self.board[0][1] = [Knight(Color.WHITE, 0, 1)]
            self.board[0][6] = [Knight(Color.WHITE, 0, 6)]
            self.board[7][1] = [Knight(Color.BLACK, 7, 1)]
            self.board[7][6] = [Knight(Color.BLACK, 7, 6)]
    
            # Place bishops
            self.board[0][2] = [Bishop(Color.WHITE, 0, 2)]
            self.board[0][5] = [Bishop(Color.WHITE, 0, 5)]
            self.board[7][2] = [Bishop(Color.BLACK, 7, 2)]
            self.board[7][5] = [Bishop(Color.BLACK, 7, 5)]
    
            # Place queens and kings
            self.board[0][3] = Queen(Color.WHITE, 0, 3)
            self.board[7][3] = Queen(Color.BLACK, 7, 3)
            self.board[0][4] = King(Color.WHITE, 0, 4)
            self.board[7][4] = King(Color.BLACK, 7, 4)

    def to_numpy(self) -> ndarray:
        """
        Converts the chessboard state to a NumPy array representation.

        Returns:
            ndarray: A (8, 8, 2) array where each cell contains the color and piece type.
        """
        result: ndarray = np.zeros((8, 8, 2), dtype=int)
        for row in range(8):
            for col in range(8):
                piece: Optional[ChessPiece] = self.board[row][col]
                result[row, col] = piece.to_numpy() if piece else np.array([0, 0])
        return result

    def piece_at(self, location: Tuple[int, int]) -> Optional[ChessPiece]:
        """
        Gets the piece at the specified location.

        Args:
            location (Tuple[int, int]): The row and column of the square.

        Returns:
            Optional[ChessPiece]: The piece at the location, or None if empty.
        """
        return self.board[location[LOCATION_ROW]][location[LOCATION_COL]]

    # includes test for check
    def get_possible_moves(self, piece: ChessPiece) -> List[Tuple[int, int]]:
        """
        Calculates potential moves for a given piece.

        Args:
            piece (ChessPiece): The chess piece to evaluate.

        Returns:
            List[Tuple[int, int]]: A list of valid destination coordinates.
        """
        potential_moves: List[Tuple[int, int]] = []
        threatened_squares: List[Tuple[int, int]] = self.get_threatened_squares(piece)

        if piece.piece_type == PieceType.PAWN:
            for square in threatened_squares:
                if self.en_passant == square:
                    potential_moves.append(square)
                else:
                    target_piece = self.board[square[0]][square[1]]
                    if target_piece and target_piece.color != piece.color:
                        potential_moves.append(square)
            forward_direction = 1 if piece.color == Color.WHITE else -1
            starting_row = 1 if piece.color == Color.WHITE else 6
            if not self.board[piece.row + forward_direction][col]:
                potential_moves.append((piece.row + forward_direction, col))
                if piece.row == starting_row and not self.board[piece.row + (2 * forward_direction)][col]:
                    potential_moves.append((piece.row + (2 * forward_direction), col))
        else:
            potential_moves.extend(threatened_squares)

        # check for castling possibilities
        if piece.piece_type == PieceType.KING:
            king_start_row = 0 if piece.color == Color.WHITE else 7
            # king cannot be in check in order to castle
            if piece.row == king_start_row and not self.is_in_check(piece.color):
                king_side = WHITE_KING_SIDE if piece.color == WHITE else BLACK_KING_SIDE
                queen_side = WHITE_QUEEN_SIDE if piece.color == WHITE else BLACK_QUEEN_SIDE
                # check that the king is not moving across check, moving into check will be tested below
                if self.check_possibilities[king_side]:
                    piece_between = self.board[king_start_row][5]
                    if not piece_between:
                        _, preview_board = self.preview_move(piece, (king_start_row, 5))
                        if preview_board and not preview_board.is_in_check(piece.color):
                            potential_moves.append((king_start_row, 6))
                if self.check_possibilities[queen_side]:
                    piece_between = self.board[king_start_row][3]
                    if not piece_between:
                        _, preview_board = self.preview_move(piece, (king_start_row, 3))
                        if preview_board and not preview_board.is_in_check(piece.color):
                            potential_moves.append((king_start_row, 2))
                    
        result: List[Tuple[int, int]] = []

        # cannot move into check
        for move in potential_moves:
            move_result, temp_board = self.preview_move(piece, move)
            if temp_board and not temp_board.is_in_check(piece.color):
                result.append(move)

        return result

    def add_move_if_valid(row: int, col: int, result: List[Tuple[int, int]]) -> None:
        """
        Adds a move to the list if it is within bounds and valid.
        """
        if 0 <= row < 8 and 0 <= col < 8:
            dest_piece = self.piece_at((row, col))
            if dest_piece is None or dest_piece.color != piece.color:
                result.append((row, col))

    # does not test for check
    def get_threatened_squares(self, piece: ChessPiece) -> List[Tuple[int, int]]:
        result: List[Tuple[int, int]] = []

        # Handle piece-specific move logic
        if piece.piece_type == PieceType.PAWN:
            if piece.col > 0:
                target_square = (piece.row + forward_direction, piece.col - 1)
                result.append(target_square)
            if piece.col < 7:
                target_square = (piece.row + forward_direction, piece.col + 1)
                result.append(target_square)
        elif piece.piece_type in [PieceType.ROOK, PieceType.BISHOP, PieceType.QUEEN]:
            move_directions = ROOK_MOVE_DIRECTIONS if piece.piece_type == PieceType.ROOK else BISHOP_MOVE_DIRECTIONS if piece.piece_type == PieceType.BISHOP else ROYAL_MOVE_DIRECTIONS
            self.get_directional_moves(piece, move_directions, result)
        elif piece.piece_type == PieceType.KNIGHT:
            self.get_knight_moves(piece, result)
        elif piece.piece_type == PieceType.KING:
            for direction in ROYAL_MOVE_DIRECTIONS:
                self.add_move_if_valid(piece.row + direction[0], piece.col + direction[1], result)

        return result

    # does not test for check
    def get_directional_moves(self, piece: ChessPiece, directions: List[Tuple[int, int]], possible_moves: List[Tuple[int, int]]) -> None:
        """
        Calculates moves for directional pieces (rook, bishop, queen).
        """
        for dr, dc in directions:
            row: int = piece.row
            col: int = piece.col
            while True:
                row += dr
                col += dc
                if 0 <= row < 8 and 0 <= col < 8:
                    dest_piece: Optional[ChessPiece] = self.piece_at((row, col))
                    if dest_piece is None:
                        possible_moves.append((row, col))
                    elif dest_piece.color != piece.color:
                        possible_moves.append((row, col))
                        break
                    else:
                        break
                else:
                    break

    # does not test for check
    def get_knight_moves(self, piece: ChessPiece, possible_moves: List[Tuple[int, int]]) -> None:
        """
        Calculates moves for knights.
        """
        for dr, dc in [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]:
            self.add_move_if_valid(piece.row + dr, piece.col + dc, possible_moves)

    def is_in_checkmate(self, color: Color) -> bool:
        # If the king is not in check, it cannot be in checkmate
        if not self.is_in_check(color):
            return False

        pieces: List[ChessPiece] = self.get_all_pieces(color)
        for piece in pieces:
            moves = self.get_possible_moves(piece)
            if len(moves) > 0:
                return False

        return True

    def get_all_pieces(self, color: Color) -> List[ChessPiece]:
        return get_all_pieces_of_type(color, piece_type = None)

    def get_all_pieces_of_type(self, color: Color, piece_type: PieceType = None) -> List[ChessPiece]:
        results: List[ChessPiece] = []
        for row in range(8):
            for col in range(8):
                piece = self.piece_at((row, col))
                # Check if the piece is a king of the correct color
                if piece.color == color and (piece_type == None or piece.piece_type == piece_type):
                    results.append(piece)
        
        return results
    
    def is_in_check(self, color: Color) -> bool:
        king: ChessPiece = self.get_all_pieces_of_type(color, PieceType.KING)[0]
        king_location: Tuple[int, int] = (king.row, king.col)
        opponent_color = Color.BLACK if color == Color.WHITE else Color.WHITE
        opponent_pieces: List[ChessPiece] = self.get_all_pieces(opponent_color)
        for piece in opponent_pieces:
            opponent_threatend_squares: List[Tuple[int, int]] = self.get_threatened_squares(piece)
            if king_location in opponent_threatened_squares:
                return True
        return False

    # does not test for check
    def preview_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> Tuple[MoveResult, Optional["ChessBoard"]]:
        board_copy = self.board.copy()
        en_passant_copy = None
        castle_possibilities_copy = self.castle_possibilities.copy()
        
        if piece.piece_type == PieceType.KING:
            castle = False
            castle_queen_side = False
            king_home_row = 0 if piece.color == Color.WHITE else 7
            
            if origin == (king_home_row, 4) and dest == (king_home_row, 2):
                castle = True
                castle_queen_side = True
            elif origin == (king_home_row, 4) and dest == (king_home_row, 6):
                castle = True

            if castle:
                opponent_pieces = self.get_all_pieces(Color.BLACK if piece.color == Color.WHITE else Color.WHITE)
                protected_squares = [(king_home_row, 4), (king_home_row, 3), (king_home_row, 2)] if castle_queen_side else [(king_home_row, 4), (king_home_row, 5), (king_home_row, 6)]
                for piece in opponent_pieces:
                    threatened_squares = self.get_threatened_squares(piece)
                    for square in protected_squares:
                        if square in threatened_squares:
                            return None
                    if castle_queen_side:
                        board_copy[king_home_row][2] = King(piece.color, king_home_row, 2)
                        board_copy[king_home_row][4] = None
                        board_copy[king_home_row][3] = Rook(piece.color, king_home_row, 3)
                        board_copy[king_home_row][0] = None
                    else:
                        board_copy[king_home_row][6] = King(piece.color, king_home_row, 6)
                        board_copy[king_home_row][4] = None
                        board_copy[king_home_row][5] = Rook(piece.color, king_home_row, 5)
                        board_copy[king_home_row][7] = None
            else:
                piece_copy: ChessPiece = piece.copy()
                piece_copy.move_to(destination[LOCATION_ROW], destination[LOCATION_COL])
                board_copy[destination[LOCATION_ROW]][destination[LOCATION_COL]] = piece_copy
                board_copy[origin[LOCATION_ROW]][origin[LOCATION_COL]] = None

            if piece.color == COLOR.WHITE:
                castle_possibilities_copy[WHITE_QUEEN_SIDE] = False
                castle_possibilities_copy[WHITE_KING_SIDE] = False
            else:
                castle_possibilities_copy[BLACK_QUEEN_SIDE] = False
                castle_possibilities_copy[BLACK_KING_SIDE] = False
        else:
            piece_copy: ChessPiece = piece.copy()
            piece_copy.move_to(destination[LOCATION_ROW], [destination[LOCATION_COL])
            board_copy[destination[LOCATION_ROW]][destination[LOCATION_COL]] = piece_copy
            board_copy[origin[LOCATION_ROW]][origin[LOCATION_COL]] = None

        if piece.piece_type == PieceType.PAWN:
            if dest == self.en_passant:
                board_copy[origin[ROW]][dest[COL]] = None
            if origin[ROW] == 1 and dest[ROW] == 3:
                en_passant_copy = (2, origin[COL])
            elif origin[ROW] == 6 and dest[ROW] == 4:
                en_passant_copy = (5, origin[COL])
        if piece.piece_type == PieceType.ROOK:
            if origin == (0, 0):
                castle_possibilities_copy[WHITE_QUEEN_SIDE] = False
            elif origin == (0, 7):
                castle_possibilities_copy[WHITE_KING_SIDE] = False
            elif origin == (7, 0):
                castle_possibilities_copy[BLACK_QUEEN_SIDE] = False
            elif origin == (7, 7):
                castle_possibilities_copy[BLACK_KING_SIDE] = False

        new_board = ChessBoard(board_copy, en_passant_copy, castle_possibilities_copy)
        opponent_color = Color.WHITE if piece.color == Color.BLACK else Color.BLACK
        move_result = MoveResult.WHITE_TURN if opponent_color == Color.BLACK else MoveResult.BLACK_TURN
        
        return (move_result, new_board)

    # includes test for check
    def make_move(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> MoveResult:
        piece: ChessPiece = self.board[origin[LOCATION_ROW]][origin[LOCATION_COL]]
        opponent_color = Color.BLACK if piece.color == Color.WHITE else Color.WHITE
        if not piece:
            return MoveResult.ERROR_NO_PIECE

        possible_moves = self.get_possible_moves(piece)
        if destination not in possible_moves:
            return MoveResult.ERROR_INVALID_MOVE

        self.board[destination[LOCATION_ROW]][destination[LOCATION_COL]] = self.board[origin[LOCATION_ROW]][origin[LOCATION_COL]]
        self.board[origin[LOCATION_ROW]][origin[LOCATION_COL]].move_to(destination[LOCATION_ROW], destination[LOCATION_COL])
        self.board[destination[LOCATION_ROW]][destination[LOCATION_COL]] = None

        # check for castling, move the rook if it is, and if the king moves, castling is no longer possible so set those values
        if piece.piece_type == PieceType.KING:
            self.en_passant = False
            king_start_row = 0 if piece.color == Color.WHITE else 7
            
            if piece.row == king_start_row:
                if piece.col == 4 and destination[LOCATION_COL] == 6:
                    self.board[piece.row][5] = self.board[piece.row][7]
                    self.board[piece.row][5].move_to(piece.row, 7)
                    self.board[piece.row][7] = None
                elif piece.col == 4 and destination[LOCATION_COL] == 2:
                    self.board[piece.row][3] = self.board[piece.row][0]
                    self.board[piece.row][3].move_to(piece.row, 0)
                    self.board[piece.row][0] = None
                    
            if piece.color == Color.WHITE:
                self.castle_possibilities[WHITE_KING_SIDE] = False
                self.castle_possibilities[WHITE_QUEEN_SIDE] = False
            else:
                self.castle_possibilities[BLACK_KING_SIDE] = False
                self.castle_possibilities[BLACK_QUEEN_SIDE] = False
        # check for en_passant, if it is an en passant capture, remove the captured piece, if it is a double first move, set the en_passant flag
        elif piece.piece_type == PieceType.PAWN:
            if destination == self.en_passant:
                self.board[origin[LOCATION_ROW]][destination[LOCATION_COL]] = None
                self.en_passant = False
            elif (origin[LOCATION_ROW] == 1 and destination[LOCATION_ROW] == 3) or (origin[LOCATION_ROW] == 6 and destination[LOCATION_ROW] == 4):
                move_direction = 1 if piece.color == Color.WHITE else -1
                self.en_passant = (piece.row + move_direction, piece.col)
            else:
                self.en_passant = False
        # if the rook moves from its original position, whether the first time or not, we set the appropriate castle flag to False
        elif piece.piece_type == PieceType.ROOK:
            self.en_passant = False
            if piece.row == 0 and piece.col == 0:
                self.castle_possibilities[WHITE_QUEEN_SIDE] = False
            elif piece.row == 0 and piece.col == 7:
                self.castle_possibilities[WHITE_KING_SIDE] = False
            elif piece.row == 7 and piece.col == 0:
                self.castle_possibilities[BLACK_QUEEN_SIDE] = False
            elif piece.row == 7 and piece.col == 7:
                self.castle_possibilities[BLACK_KING_SIDE] = False

        if self.is_in_checkmate(opponent_color):
            return MoveResult.BLACK_WINS if opponent_color == Color.WHITE else MoveResult.WHITE_WINS
        if self.is_in_check(opponent_color):
            return MoveResult.WHITE_TURN_CHECK if opponent_color == Color.WHITE else MoveResult.BLACK_TURN_CHECK
        return MoveResult.WHITE_TURN if opponent_color == Color.WHITE else MoveResult.BLACK_TURN

## 2.4 The Game

Here we have the core logic that figures out valid moves, turn tracking, the UI, and anything else that got tacked on.

So let's go through take_turn() a little closer:

1. we make sure we have a valid starting piece.
2. we see if that piece can move there. Our board has no idea of whether en passant or castling is possible in this case since it has no history. It will include castling and en passant if potentially possible and leave it to us here to perform final validation.
3. If this move is a castle, we check to make sure it's valid by comparing it to the variables we track in step 8.
4. If this move is an en passant, we compare our destination to the en_passant_square we track in step 7.
5. Now that it's been validated as a legal move, we tell our board to go ahead and make the move.
6. If that didn't return an error because it moved into check, then we now have our updated board which we add to our history numpy array. This array is what we'll be using to chop up into data for training and testing sets.
7. If this is a pawn moving ahead 2, then it is vulnerable next turn to en passant and set the skipped over square to our en_passant_square, otherwise en passant is not possible next turn so set it to None.
8. And finally, moving a king or rook will invalidate future castlings of the appropriate type(s).

In [ ]:
class ChessGame:
    def __init__(self):
        self.board: ChessBoard = ChessBoard()
        self.history: ndarray = np.zeros((1, 8, 8, 2), dtype=int)
        self.history[0] = self.board.to_numpy()
        self.player_turn: Color = Color.WHITE
            
        # if a pawn moves 2, they are vulnerable to en passant, but only after just moving
        self.en_passant_square: Optional[Tuple[int, int]] = None
        
        # if the pieces move, these change, this is just faster than calculating it each time
        self.black_king_castle: bool = True
        self.black_queen_castle: bool = True
        self.white_king_castle: bool = True
        self.white_queen_castle: bool = True
    
    def take_turn(self, origin: Tuple[int, int], destination: Tuple[int, int]) -> MoveResult:
        origin_piece: Optional[ChessPiece] = self.board.piece_at(origin)
        destination_piece: Optional[ChessPiece] = self.board.piece_at(destination)
        
        # make sure we have a valid piece
        if not origin_piece:
            print('No piece at that position to move', origin, destination)
            return MoveResult.ERROR_NO_PIECE
        if not origin_piece.color == self.player_turn:
            print('Not your piece', origin, destination)
            return MoveResult.ERROR_WRONG_COLOR
        
        # the list of all possible moves, including en passant and castling
        destination_list: List[Tuple[int, int]] = self.board.get_possible_moves(origin_piece)
        
        # that piece cannot reach the destination in any scenario
        if destination not in destination_list:
            print('That piece cannot move there', origin, destination)
            return MoveResult.ERROR_CANNOT_REACH
        
        # if it is a king, check if it's a castle
        if origin_piece.piece_type == PieceType.KING and is_castle(origin, destination):
            # piece where the rook will land
            middlePiece: Optional[ChessPiece] = self.board.piece_at((origin[0], (origin[1] + destination[1]) / 2))
            # piece beside the rook if queen side castling
            internal_piece: Optional[ChessPiece] = None if destination[1] == 6 else self.board.piece_at((origin[0], 1))
                
            # castling is only possible if the destination and intervening pieces are empty
            if middle_piece or destination_piece or internal_piece:
                print('Illegal castle, piece in the way')
                return MoveResult.ERROR_CANNOT_REACH
            
            # check if that particular castle is possible
            if origin == (0, 4) and dest == (0, 2):
                if not self.white_king_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (0, 4) and dest == (0, 6):
                if not self.white_queen_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (7, 4) and dest == (7, 2):
                if not self.black_king_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
            if origin == (7, 4) and dest == (7, 6):
                if not self.black_queen_castle:
                    print('Illegal castle', origin, destination)
                    return MoveResult.ERROR_CANNOT_REACH
        
        # if it is an en passant, validate it
        if origin_piece.piece_type == PieceType.PAWN and origin_piece.can_capture(destination) and self.board.piece_at(destination) is None:
            if self.en_passant_square != destination:
                print('Illegal move, no en passant possible and no piece to take', origin, destination)
                return MoveResult.ERROR_CANNOT_REACH
        
        # otherwise make the move
        move_result, new_board = self.board.make_move(origin, destination)
                
        # if it's moving into check, report error
        if move_result == MoveResult.ERROR_CHECK:
            print('Invalid move, that would put you in check', origin, destination)
            return MoveResult.ERROR_CHECK
        
        # update the move history
        new_board = new_board.reshape(1, 8, 8, 2)
        self.history = np.concatenate((self.history, new_board), axis=0)
            
        # track en passant, only possible after a pawn first moves 2 spaces and is the space that was jumped over, otherwise clear it
        if origin_piece.piece_type == PieceType.PAWN:
            if origin_piece.color == Color.WHITE and origin[0] == 1 and destination[0] == 3:
                self.en_passant_square = (2, origin[1])
            elif origin_piece.color == Color.BLACK and origin[0] == 6 and destination[0] == 4:
                self.en_passant_square = (5, origin[1])
            else:
                self.en_passant_square = None
        else:
            self.en_passant_square = None
            
        # rook and king moves make castling unavailable
        if origin == (0, 4):
            self.white_king_castle = False
            self.white_queen_castle = False
        elif origin == (0, 0):
            self.white_queen_castle = False
        elif origin == (0, 7):
            self.white_king_castle = False
        elif origin == (7, 4):
            self.black_king_castle = False
            self.black_queen_castle = False
        elif origin == (7, 0):
            self.black_queen_castle = False
        elif origin == (7, 7):
            self.black_king_castle = False
                
        # return the move result
        return move_result
    
    def take_turn_uci(self, uci_move: str) -> MoveResult:
        pass
    
    def display_board(self) -> None:
        pass

Now to start testing and debugging it:

In [ ]:
newGame = ChessGame()